In [29]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Cesta ke skriptům
sys.path.append(os.path.abspath("../scripts"))
from utils import Normalizer, create_multifeature_lstm_dataset

In [30]:
# Trénovací parametry
WINDOW_SIZE = 20
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
PATIENCE = 5

# Načti předzpracovaná data
import pickle

with open("../data/preprocessed_datasets.pkl", "rb") as f:
    vsechny_datasety = pickle.load(f)

# Vyber jednu firmu (např. NVDA)
vybrany = [d for d in vsechny_datasety if d['ticker'] == 'NVDA'][0]

x_train = torch.tensor(vybrany['x_train'], dtype=torch.float32)
y_train = torch.tensor(vybrany['y_train'], dtype=torch.float32)
x_val = torch.tensor(vybrany['x_val'], dtype=torch.float32)
y_val = torch.tensor(vybrany['y_val'], dtype=torch.float32)

In [31]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

# Inicializace modelu
input_size = x_train.shape[2]
model = LSTMModel(input_size=input_size, hidden_size=64, num_layers=2, dropout=0.3)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=BATCH_SIZE)

train_losses, val_losses = [], []
best_val_loss = float("inf")
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        pred = model(xb).squeeze()
        loss = loss_fn(pred, yb.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            pred = model(xb).squeeze()
            loss = loss_fn(pred, yb.view(-1))
            val_loss += loss.item()
    val_loss /= len(val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        print("Early stopping – validace se nezlepšuje")
        break

Epoch 1/50 - Train Loss: 0.0032 - Val Loss: 0.0168
Epoch 2/50 - Train Loss: 0.0002 - Val Loss: 0.0187
Epoch 3/50 - Train Loss: 0.0001 - Val Loss: 0.0194
Epoch 4/50 - Train Loss: 0.0001 - Val Loss: 0.0195
Epoch 5/50 - Train Loss: 0.0001 - Val Loss: 0.0224
Epoch 6/50 - Train Loss: 0.0001 - Val Loss: 0.0158
Epoch 7/50 - Train Loss: 0.0001 - Val Loss: 0.0160
Epoch 8/50 - Train Loss: 0.0001 - Val Loss: 0.0168
Epoch 9/50 - Train Loss: 0.0001 - Val Loss: 0.0180
Epoch 10/50 - Train Loss: 0.0001 - Val Loss: 0.0110
Epoch 11/50 - Train Loss: 0.0001 - Val Loss: 0.0158
Epoch 12/50 - Train Loss: 0.0001 - Val Loss: 0.0096
Epoch 13/50 - Train Loss: 0.0001 - Val Loss: 0.0216
Epoch 14/50 - Train Loss: 0.0001 - Val Loss: 0.0203
Epoch 15/50 - Train Loss: 0.0001 - Val Loss: 0.0108
Epoch 16/50 - Train Loss: 0.0001 - Val Loss: 0.0208
Epoch 17/50 - Train Loss: 0.0001 - Val Loss: 0.0102
Early stopping – validace se nezlepšuje
